In [3]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np
import json

def create_interactive_map_data(towns, census_tracts_clipped, dams, mill_pond_buffer_gdf, stump_pond_buffer_gdf, land_use_2016=None):
    """
    Generates data for an interactive Leaflet map, including GeoJSON and styling information.

    Args:
        towns (GeoDataFrame): Town boundaries.
        census_tracts_clipped (GeoDataFrame): Census tracts clipped to town boundaries.
        dams (GeoDataFrame): Dam locations (optional).
        mill_pond_buffer_gdf (GeoDataFrame): Buffer around Mill Pond (optional).
        stump_pond_buffer_gdf (GeoDataFrame): Buffer around Stump Pond (optional).
        land_use_2016 (GeoDataFrame): Land use data (optional).

    Returns:
        dict: A dictionary containing GeoJSON data and styling information.
    """

    map_data = {}

    # 1. Town Boundaries (GeoJSON)
    map_data['towns'] = json.loads(towns.to_json())

    # 2. Census Tracts with Population Density (GeoJSON & Styling)
    if census_tracts_clipped is not None:
        map_data['census_tracts'] = json.loads(census_tracts_clipped.to_json())

        # Define color stops for population density (adjust as needed)
        density_stops = [
            (0, '#ffffd9'),   # Lightest color for lowest density
            (100, '#c7e9b4'),
            (500, '#7fcdbb'),
            (1000, '#41b6c4'),
            (2000, '#1d91c0'),
            (3000, '#225ea8'),
            (4000, '#253494'),
            (float('inf'), '#081d58')  # Darkest color for highest density
        ]
        map_data['census_tracts_style'] = {
            'fillColor': {'property': 'population_density', 'stops': [
                {'lte': stop[0], 'color': stop[1]} for stop in density_stops
            ]},
            'weight': 0.5,
            'opacity': 1,
            'color': 'grey',
            'fillOpacity': 0.7
        }

    # 3. Land Use (Optional - GeoJSON & Styling)
    if land_use_2016 is not None:
        map_data['land_use'] = json.loads(land_use_2016.to_json())
        # Example styling - customize this based on your land use categories
        map_data['land_use_style'] = {
            'fillColor': {'property': 'SCLANDUSE', 'type': 'categorical', 'stops': [
                {'value': 1, 'color': 'lightcoral'},  # Developed
                {'value': 7, 'color': 'lightgreen'},  # Parks
                {'value': 13, 'color': 'skyblue'},   # Water
                # Add more stops for other land use categories
            ]},
            'weight': 0.5,
            'color': 'black',
            'fillOpacity': 0.7
        }

    # 4. Dams (GeoJSON & Styling)
    if dams is not None:
        map_data['dams'] = json.loads(dams.to_json())
        map_data['dams_style'] = {
            'radius': 6,
            'fillColor': 'purple',
            'color': '#000',
            'weight': 1,
            'opacity': 1,
            'fillOpacity': 0.8
        }

    # 5. Buffers (GeoJSON & Styling)
    if mill_pond_buffer_gdf is not None:
        map_data['mill_pond_buffer'] = json.loads(mill_pond_buffer_gdf.to_json())
        map_data['mill_pond_buffer_style'] = {
            'color': 'blue',
            'weight': 2,
            'fillOpacity': 0.1
        }
    if stump_pond_buffer_gdf is not None:
        map_data['stump_pond_buffer'] = json.loads(stump_pond_buffer_gdf.to_json())
        map_data['stump_pond_buffer_style'] = {
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0.1
        }

    return map_data